In [1]:
from qdrant_haystack.document_stores import QdrantDocumentStore
from haystack.nodes import EntityExtractor
from haystack.pipelines import Pipeline
from haystack.nodes import PreProcessor, BM25Retriever, FARMReader
import torch

/Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
with open('articles_db.pkl', 'rb') as f:
    document_store = pickle.load(f)

In [3]:
document_store.get_all_documents()[0]

<Document: {'content': 'With a population of around 175,000, Trondheim is Norway’s third city, but the pace of life here is slow and easy, and the main sights are best appreciated in leisurely fashion over a couple of days. Genial and eminently likeable, Trondheim is a pleasant place to wave goodbye to city life before heading for the wilds of the north.\n\nTrondheim is also the capital of the Trøndelag province, whose sweeping valleys are – by Norwegian standards at least – very fertile and profitable: indeed the region’s landowners acted as a counterweight to the power of the south for hundreds of years and, when the country regained its independence in 1905, it seemed logical for the new dynasty to hold their coronations here in Trondheim.\n\nAccommodation\n\nAccommodation is plentiful in Trondheim, with a choice of private rooms, two hostels, and a selection of reasonably priced hotels. What’s more, most of the more appealing places are dotted round the city centre, which is precis

In [4]:
from haystack.nodes import EmbeddingRetriever
retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    use_gpu=True,
    devices=[torch.device("mps")],
)

/Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You seem to be using sentence-transformers/multi-qa-mpnet-base-dot-v1 model with the cosine function instead of the recommended dot_product. This can be set when initializing the DocumentStore


In [5]:
reader =  FARMReader(model_name_or_path="deepset/roberta-base-squad2",use_gpu=True,devices=[torch.device("mps")])

query_pipeline = Pipeline()
query_pipeline.add_node(component=retriever, name="retriever", inputs=["Query"])
query_pipeline.add_node(component=reader,name="reader",inputs=["retriever"])

In [6]:
from haystack.utils import print_answers
place = "Trondheim"
results = query_pipeline.run(query=f"Attractions of {place}:",
    params={
        "filters": {
            "entity_words": [place]}
        })
print_answers(results)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]/Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/transformers/models/mpnet/modeling_mpnet.py:1064: UserWarning: torch.cumsum supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:264.)
  incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]/Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/haystack/modeling/model/prediction_head.py:467: UserWarning: The operator 'aten::tril_indices' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  indices = torch.tril_indices(max_seq_len, max_seq_len, offset=-1, device=start_end_matrix.device)


'Query: Attractions of Trondheim:'
'Answers:'
[   <Answer {'answer': 'Nidaros Cathedral', 'type': 'extractive', 'score': 0.6130564212799072, 'context': 'highlights during a long weekend. Explore top attractions like the Nidaros Cathedral in Trondheim, the UNESCO-listed Bryggen wharf in Bergen, the Arct', 'offsets_in_document': [{'start': 210, 'end': 227}], 'offsets_in_context': [{'start': 67, 'end': 84}], 'document_ids': ['4ebc9b86d15bad77f78f558a6d7900bb'], 'meta': {'url': 'https://www.visitnorway.com/places-to-go/city-break/', 'source_url': 'https://www.visitnorway.com', 'lang': 'en', 'authors': [], 'publish_date': None, 'movies_url': [], 'top_image_url': 'https://assets.simpleviewcms.com/simpleview/image/upload/c_limit,h_1200,q_75,w_1200/v1/clients/norway/woman_at_aksla_viewpoint_alesund_fjord_norway_photo_samuel_taipale_visitnorway_com_2_1_d8fe3b3d-9c01-40ed-afe6-7a2c5549d79e.jpg', 'article_keywords': ['best', 'city', 'ålesund', 'long', 'trondheim', 'bergen', 'wharf', 'tromsø', 'we